In [3]:
# Load Packages
from __future__ import unicode_literals, print_function

import plac #  wrapper over argparse
import random
from pathlib import Path
import spacy
from tqdm import tqdm, tqdm_notebook # loading bar 
import pandas as pd
import re
from pprint import pprint
from nltk.tokenize import sent_tokenize
import numpy as np
import unicodedata
from spacy.util import minibatch, compounding

In [4]:
# Função para substituir acentos

def strip_accents(text):
    try:
        text = unicode(text, 'utf-8')
    except (TypeError, NameError): # unicode is a default on python 3 
        pass
    text = unicodedata.normalize('NFD', text)
    text = text.encode('ascii', 'ignore')
    text = text.decode("utf-8")
    
    return str(text)

# Abrindo e gerando Datasets

### Entidade logradouro

In [5]:
# Obtendo dados crus de endereço

dataset = pd.read_csv("../Datasets/201906AGENCIAS.CSV")
dset = dataset.iloc[:-2,4:10].values  # tipo nparray object
pd_dset = pd.DataFrame(dset) # caso queira visualizar

pd_dset['Join'] = pd_dset[pd_dset.columns[0:]].apply(
    lambda x: ';'.join(x.dropna().astype(str)),
    axis=1
)
pd_dset.head(5)

,0,1,2,3,4,5,Join
0,"R.GUILHERME MOREIRA,315","SUBLOJA,LOJA E 2.ANDAR ...",CENTRO,69005-300,MANAUS ...,AM,"R.GUILHERME MOREIRA,315 ;SUBLO..."
1,"AV.PRES.VARGAS,248",1.E 2.ANDARES ...,CAMPINA,66010-900,BELEM ...,PA,"AV.PRES.VARGAS,248 ;1.E 2..."
2,"R.QUINZE DE NOVEMBRO,195",...,CENTRO,11010-908,SANTOS ...,SP,"R.QUINZE DE NOVEMBRO,195 ; ..."
3,"PCA.DAS QUATRO JORNADAS,11",MEZANINO ...,CENTRO,28010-000,CAMPOS DOS GOYTACAZES ...,RJ,"PCA.DAS QUATRO JORNADAS,11 ;MEZAN..."
4,"SEXTA AVENIDA,600",SECRETARIA DA EDUCACAO-TERREO ...,CAB,41745-002,SALVADOR ...,BA,"SEXTA AVENIDA,600 ;SECRE..."


In [16]:
# Tratando endereço completo do DataFrame
dset = np.array(pd_dset)
end_lista = []

for i in range(len(dset)):
    str_raw = dset[i][6]
    str_tratada = re.sub(r'[ ]{2,}', "",str_raw) # Tirando espaços excedentes no final do endereço
    str_tratada = re.sub(r'[;]{1,}', "; ", str_tratada) # Para complementos vazios, para não ter 2 ";"
    str_tratada = str_tratada.lower()
    str_tratada = str_tratada.replace(",", ", ")
    str_tratada = str_tratada.replace(".", " ")
    str_tratada = strip_accents(str_tratada)
    str_tratada = re.sub(r'[ ]{2,}', " ", str_tratada)

    end_lista.append(str_tratada)

end_lista[:3]

['r guilherme moreira, 315; subloja, loja e 2 andar; centro; 69005-300; manaus; am',
 'av pres vargas, 248; 1 e 2 andares; campina; 66010-900; belem; pa',
 'r quinze de novembro, 195; centro; 11010-908; santos; sp']

In [17]:
# Lógica para extrair posição do logradouro:
# Dividir a String inteira por ";", pegar o len do primeiro split
# len de LOGRA é de 0 até len do primeiro split

print(end_lista[0])
split = end_lista[0].split(";")
print(split)
print(split[0])
print(len(split[0]))

r guilherme moreira, 315; subloja, loja e 2 andar; centro; 69005-300; manaus; am
['r guilherme moreira, 315', ' subloja, loja e 2 andar', ' centro', ' 69005-300', ' manaus', ' am']
r guilherme moreira, 315
24


In [18]:
# Extraindo iob com entidade apenas de logradouro e fazendo mais tratamentos de texto

iob = []

for i in range(len(end_lista)):
    split = end_lista[i].split(";")
    len_logra = len(split[0])
    iob_dict = {"entities": [(0, len_logra-1,'LOGRA')]}
    end_lista[i] = end_lista[i].replace(";", " ")
    end_lista[i] = end_lista[i].replace("  ", " ")
#     end_lista[i] = end_lista[i].replace(",", " ")
    end_lista[i] = re.sub(r'^[ ]+', "", end_lista[i])
    
    tupla = (end_lista[i], iob_dict)
    iob.append(tupla)

FULL_DATA = iob[:2000]

In [19]:
print(FULL_DATA[0])
print(FULL_DATA[1])
print(FULL_DATA[2])
print(FULL_DATA[15])

('r guilherme moreira, 315 subloja, loja e 2 andar centro 69005-300 manaus am', {'entities': [(0, 23, 'LOGRA')]})
('av pres vargas, 248 1 e 2 andares campina 66010-900 belem pa', {'entities': [(0, 18, 'LOGRA')]})
('r quinze de novembro, 195 centro 11010-908 santos sp', {'entities': [(0, 24, 'LOGRA')]})
('r sao bento, 483 centro 01011-100 sao paulo sp', {'entities': [(0, 15, 'LOGRA')]})


In [20]:
# Criação da base de teste e treinamento

n_test= 0.1 # Porcentagem para base de teste
test_n = round(len(FULL_DATA) * n_test)

# Divisao em Train Test Val

def gerador_bases(dataset, n):
    indices_random = random.sample(range(0,len(dataset)-1), n)
    base_teste_n = []
    base_treinamento_n = []
    
    for i in range(n):
        base_teste_n.append(dataset[indices_random[i]])

    for j in range(len(dataset)):
        if(j not in indices_random):
            base_treinamento_n.append(dataset[j])
            
    return base_teste_n, base_treinamento_n


base_teste, base_treinamento = gerador_bases(FULL_DATA, test_n)

random.shuffle(base_treinamento)
random.shuffle(base_teste)

print("Treinamento: " + str(len(base_treinamento)), "\nTeste: " + str(len(base_teste)), "\nTotal: " + str(len(FULL_DATA)))

Treinamento: 1800 
Teste: 200 
Total: 2000


In [21]:
for i in range(10):
    print(base_treinamento[i])

('r alfredo backer, 536 loja a alcantara 24452-005 sao goncalo rj', {'entities': [(0, 20, 'LOGRA')]})
('av miguel varlez, 136 centro 11660-650 caraguatatuba sp', {'entities': [(0, 20, 'LOGRA')]})
('av pres vargas, 1184 terreo centro 99670-000 ronda alta rs', {'entities': [(0, 19, 'LOGRA')]})
('pca mons jose candido, 156 centro 63870-000 boa viagem ce', {'entities': [(0, 25, 'LOGRA')]})
('r pres backer, 124 icarai 24220-041 niteroi rj', {'entities': [(0, 17, 'LOGRA')]})
('av sete de setembro, 285 centro 84430-000 imbituva pr', {'entities': [(0, 23, 'LOGRA')]})
('r cel otavio magalhaes, 1 centro 77330-000 arraias to', {'entities': [(0, 24, 'LOGRA')]})
('pca jesuino braga, s/n centro 39320-000 ubai mg', {'entities': [(0, 21, 'LOGRA')]})
('r gal joao antonio, 1232 centro 97420-000 sao vicente do sul rs', {'entities': [(0, 23, 'LOGRA')]})
('pca epitacio pessoa, 35 centro 58360-000 itabaiana pb', {'entities': [(0, 22, 'LOGRA')]})


# Carregando o modelo

In [22]:
# Define our variables

model = None
output_dir=Path(".")
n_iter= 100 # número de épocas
batch_size = 32

In [23]:
# Setting up the pipeline and entity recognizer.
if model is not None:
    nlp = spacy.load(model)  # load existing spacy model
    print("Loaded model '%s'" % model)
else:
    lan = 'pt'
    nlp = spacy.blank(lan)  # create blank Language class
    print("Created blank '%s' model" % lan)
    
if 'ner' not in nlp.pipe_names:
    ner = nlp.create_pipe('ner')
    nlp.add_pipe(ner)
    print('Added new NER')
else:
    ner = nlp.get_pipe('ner')
    print('Got an old NER')

Created blank 'pt' model
Added new NER


In [24]:
# create the built-in pipeline components and add them to the pipeline
    # nlp.create_pipe works for built-ins that are registered with spaCy
if 'ner' not in nlp.pipe_names:
    ner = nlp.create_pipe('ner')
    nlp.add_pipe(ner, last=True)
# otherwise, get it so we can add labels
else:
    ner = nlp.get_pipe('ner')

## Treinamento

In [25]:
print("Batch size: ", batch_size)
print("Épocas: ", n_iter)
print()

# add labels
for _, annotations in base_treinamento:
    for ent in annotations.get('entities'):
        ner.add_label(ent[2])


# get names of other pipes to disable them during training
other_pipes = [pipe for pipe in nlp.pipe_names if pipe != 'ner']
with nlp.disable_pipes(*other_pipes):  # only train NER
    optimizer = nlp.begin_training()
    for itn in range(n_iter):
        random.shuffle(base_treinamento)
        losses = {}
        batches = minibatch(base_treinamento, size=batch_size)
        
        for batch in batches:
            texts, annotations = zip(*batch)
            try:
                nlp.update(texts, annotations, sgd=optimizer, drop=0.2,
                       losses=losses)
            except:
                pass     

        print(itn+1, ' Losses', losses)        

Batch size:  32
Épocas:  100

1  Losses {'ner': 2435.022968630853}
2  Losses {'ner': 813.3478246242817}
3  Losses {'ner': 699.3845468441936}
4  Losses {'ner': 493.97414442750977}
5  Losses {'ner': 352.7640323690594}
6  Losses {'ner': 282.43375146954423}
7  Losses {'ner': 253.60709409248008}
8  Losses {'ner': 331.67538340570957}
9  Losses {'ner': 217.42224349688374}
10  Losses {'ner': 186.22437523244457}
11  Losses {'ner': 245.84279787426843}
12  Losses {'ner': 263.14719206791546}
13  Losses {'ner': 236.48850227981185}
14  Losses {'ner': 272.5537822099532}
15  Losses {'ner': 578.3159944579454}
16  Losses {'ner': 187.0266775181807}
17  Losses {'ner': 318.3468696755638}
18  Losses {'ner': 451.4351607002218}
19  Losses {'ner': 1573.3701293687984}
20  Losses {'ner': 1173.192375302517}
21  Losses {'ner': 307.4629575553915}
22  Losses {'ner': 1627.3042210866274}
23  Losses {'ner': 371.37306874492845}
24  Losses {'ner': 98.09769787657342}
25  Losses {'ner': 1232.5004165957976}
26  Losses {'ner

## Teste

In [33]:
for each in base_treinamento[:10]:
    print(each)

('av cel martiniano, 417 centro 59300-000 caico rn', {'entities': [(0, 21, 'LOGRA')]})
('r gal bocaiuva, 2 centro 23815-310 itaguai rj', {'entities': [(0, 16, 'LOGRA')]})
('av landulfo alves, 46 centro 48602-900 paulo afonso ba', {'entities': [(0, 20, 'LOGRA')]})
('av mario melo, 400-b centro 53610-605 igarassu pe', {'entities': [(0, 19, 'LOGRA')]})
('r pref jose vieira, 140 centro 36780-000 astolfo dutra mg', {'entities': [(0, 22, 'LOGRA')]})
('av tiradentes, 119 centro 57360-000 girau do ponciano al', {'entities': [(0, 17, 'LOGRA')]})
('r joao de paiva, s/n centro 59182-000 monte alegre rn', {'entities': [(0, 19, 'LOGRA')]})
('r mariz e barros, 13 loja a pca da bandeira 20270-005 rio de janeiro rj', {'entities': [(0, 19, 'LOGRA')]})
('av mal floriano peixoto, 1750 em frente a pca da matriz centro 68600-000 braganca pa', {'entities': [(0, 28, 'LOGRA')]})
('r pe feijo, 423 predio centro 95190-000 sao marcos rs', {'entities': [(0, 14, 'LOGRA')]})


In [34]:
# Com base de teste
for text, _ in base_teste:
    doc = nlp(text)
    
    print(text)
    print('Entities', [(ent.text, ent.label_) for ent in doc.ents])
#     print('Tokens', [(t.text, t.ent_type_, t.ent_iob) for t in doc])
    print()

r quinze de novembro, 160 centro 99770-000 aratiba rs
Entities [('r quinze de novembro, 160 centro 99770-000', 'LOGRA')]

av carlos hugueney, 876 centro 78780-000 alto araguaia mt
Entities [('av carlos hugueney, 876 centro', 'LOGRA')]

avquinze de novembro, 116 centro 06850-100 itapecerica da serra sp
Entities [('avquinze de novembro, 116 centro 06850-100', 'LOGRA')]

av siqueira campos, 169 predio centro 55665-000 camocim de sao felix pe
Entities [('av siqueira campos, 169 predio', 'LOGRA')]

av rui barbosa, 75 qd 38, lote 5 centro 75940-000 edeia go
Entities [('av rui barbosa, 75 qd 38', 'LOGRA')]

r cel alexandrino, 860 centro 62800-000 aracati ce
Entities [('r cel alexandrino, 860 centro', 'LOGRA')]

r agripio teodorio soares, 157 centro 62260-000 reriutaba ce
Entities [('r agripio teodorio soares, 157', 'LOGRA')]

r dr orlando teixeira, 29 terreo centro 48410-000 cicero dantas ba
Entities [('r dr orlando teixeira, 29 terreo', 'LOGRA')]

r mal deodoro, 451 centro 95760-000 sao seba

av juscelino kubitschek, 3543 v portes 85865-000 foz do iguacu pr
Entities [('av juscelino kubitschek, 3543 v', 'LOGRA')]

pca euvaldo lodi, 35 barra da tijuca 22640-010 rio de janeiro rj
Entities [('pca euvaldo lodi, 35 barra', 'LOGRA')]

av goncalves dias, 1052 centro 65110-000 sao jose de ribamar ma
Entities [('av goncalves dias, 1052 centro', 'LOGRA')]

av mal candido rondon, 97 centro 85840-000 ceu azul pr
Entities [('av mal candido rondon, 97 centro', 'LOGRA')]

r jeronimo vervloet, 178 centro 29650-000 santa teresa es
Entities [('r jeronimo vervloet, 178 centro', 'LOGRA')]

r dez, 150 ed do forum setor oeste 74120-020 goiania go
Entities [('r dez, 150 ed do forum', 'LOGRA')]

r cel vidal ramos, 349 centro 89520-000 curitibanos sc
Entities [('r cel vidal ramos, 349 centro', 'LOGRA')]

pca mario dourado sobrinho, 100 centro 44900-000 irece ba
Entities [('pca mario dourado sobrinho, 100 centro', 'LOGRA')]

av rodolfo guimaraes, 673 centro 17380-000 brotas sp
Entities [('av rodolfo 

r cel jose dulce, 234 centro 78200-000 caceres mt
Entities [('r cel jose dulce, 234 centro', 'LOGRA')]

r pres vargas, 29 terreo centro 29400-000 mimoso do sul es
Entities [('r pres vargas, 29 terreo', 'LOGRA')]

r assis brasil, 67 centro 95185-000 carlos barbosa rs
Entities [('r assis brasil, 67 centro', 'LOGRA')]



In [15]:
# Função para tratamento de texto de entrada

def regex(str_):
    ph_tratada = str_.replace("\"", "")
    ph_tratada = ph_tratada.replace("-", " ")
    ph_tratada = ph_tratada.replace("|", "")
    ph_tratada = ph_tratada.replace(",", "")
    ph_tratada = ph_tratada.replace(":", "")
    ph_tratada = ph_tratada.replace(".", "")
    ph_tratada = ph_tratada.replace("  ", "")
    ph_tratada = ph_tratada.lower()
    ph_tratada = re.sub(r'[\n]+', " ", ph_tratada)
    ph_tratada = re.sub(r'^[ ]+', "", ph_tratada)

    ph_tratada = strip_accents(ph_tratada
                              )    
    return ph_tratada

In [ ]:
# Com texto IPTU

ph ="""
        "PREFEITURA DE Certidão de Dados Cadastrais do Imóvel - IPTU 2017\n\nSÃO PAULO\n\nFAZENDA",
        "Cadastro do Imóvel: 067.061.0048-1",
        "| Local do Imóvel:",
        "| AVJACANA, 764- AP 104 E VG",
        "BLOCO 1 UP LIFE SAO PAULO CEP 02273-001\nImóvel localizado na 22 Subdivisão da Zona Urbana",
        "| Endereço para entrega da notificação:",
        "| AVJACANA, 764- AP 104 E VG",
        "| BLOCO1 UPLIFE SAO PAULO CEP 02273-001",
        "| Contribuinte(s):",
        "| AGUA DAS FLORES EMPREENDIMENTOS IMOB LTDA",
        "| Dados cadastrais do terreno:",
        "| Área incorporada (m?): 1.669 Testada (m): 0,00",
        "| Área não incorporada (m?): 0 Fração ideal: 0,0024",
        "| Área total (m?): 7.669",
        "| Dados cadastrais da construção:",
        "| Área construída (m?):\n\n88\n\nPadrão da construção:\n\n2-C",
        "Área ocupada pela construção (m?): 5.134 Uso: residência",
        "| Ano da construção corrigido: 2016",
        "| Valores de m? (R$):",
        "| - de terreno: 700,00",
        "| - da construção: 1.508,00",
        "| Valores para fins de cálculo do IPTU (R$):",
        "| - da área incorporada: 23.675,00",
        "- da área não incorporada: 0,00",
        "| - da construção: 131.377,00",
        "|| Base de cálculo do IPTU: 155.052,00",
        "Ressalvado o direito da Fazenda Pública do Município de São Paulo atualizar os dados constantes do Cadastro\nImobiliário Fiscal, apurados ou verificados a qualquer tempo, inclusive em relação ao exercício abrangido por\nesta certidão, a Secretaria Municipal da Fazenda CERTIFICA que os dados cadastrais acima foram utilizados no\nlançamento do Imposto Predial e Territorial Urbano do imóvel do exercício de 2017.",
        "Certidão expedida via Internet - Portaria SF nº 008/2004, de 28/01/2004.",
        "A autenticidade desta certidão poderá ser confirmada, até o dia 05/02/2018, em",
        "http://www .prefeitura.sp.gov.br/cidade/secretarias/financas/servicos/certidoes/",
        "| Data de Emissão: 07/11/2017",
        "| Número do Documento: 2.2017.001599486-1",
        "|| Solicitante: PAULO AFONSO DECICINO (CPF 892.407.408-30)"
    """      


In [ ]:
ph_tratada = regex(ph)
print(ph_tratada)

In [ ]:
doc_ph = nlp(ph_tratada)
print('Entities', [(ent.text, ent.label_) for ent in doc_ph.ents])

In [21]:
# Com texto qualquer

phrase = regex("QNE 20, 12 leonardo a a ")

doc = nlp(phrase)
print('Entities', [(ent.text, ent.label_) for ent in doc.ents])
print('Tokens', [(t.text, t.ent_type_, t.ent_iob) for t in doc])

Entities [('qne 20 12 leonardo', 'LOGRA')]
Tokens [('qne', 'LOGRA', 3), ('20', 'LOGRA', 1), ('12', 'LOGRA', 1), ('leonardo', 'LOGRA', 1), ('a', '', 2), ('a', '', 2)]
